<div align="center">

# 🧠 → 🧠 EmbeddingAdapters *Quickstart*

<img src="https://raw.githubusercontent.com/PotentiallyARobot/EmbeddingAdapters/main/embeddingadapters.png" alt="EmbeddingAdapters Logo" width="400">

*Bridge embedding spaces. Use adapters, not hacks.*

</div>

---

This notebook will get you up and running with `embedding-adapters` in minutes. You'll learn how to:

1. Install the library
2. Load a pre-trained adapter
3. Translate embeddings from a local model into a target embedding space
4. (Optional) Score embedding quality and detect out-of-distribution inputs

---

## 1. Installation

Install the required packages:

In [1]:
!pip install -q embedding-adapters sentence-transformers torch numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 4.1 MB/s eta 0:00:00


## 2. Setup & Imports

In [2]:
import os
import time
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from embedding_adapters import EmbeddingAdapter

# Detect device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


### (Optional) Hugging Face Token

Some adapters require a Hugging Face token. Set it here if needed:

```python
os.environ['HUGGINGFACE_TOKEN'] = 'your_token_here'
```

Or create one at: https://huggingface.co/settings/tokens

In [3]:
# Uncomment and set your token if needed:
# os.environ['HUGGINGFACE_TOKEN'] = 'hf_xxxxxxxxxxxxx'

---

## 3. Basic Usage: Translate Embeddings

In this example, we'll:
1. Load a **local source model** (`all-MiniLM-L6-v2`)
2. Load a **pre-trained adapter** that maps to OpenAI's `text-embedding-3-small` space
3. Generate embeddings and translate them into the target space

### 3.1 Load the Source Model

In [4]:
SOURCE_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

src_model = SentenceTransformer(SOURCE_MODEL, device=device)
print(f"Loaded source model: {SOURCE_MODEL}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loaded source model: sentence-transformers/all-MiniLM-L6-v2


### 3.2 Load the Adapter

In [5]:
TARGET_MODEL = "openai/text-embedding-3-small"

adapter = EmbeddingAdapter.from_registry(
    source=SOURCE_MODEL,
    target=TARGET_MODEL,
    flavor="large",  # Options may include: "small", "medium", "large"
    device=device,
    # huggingface_token=os.environ.get('HUGGINGFACE_TOKEN')  # Uncomment if needed
)

print(f"Loaded adapter: {SOURCE_MODEL} → {TARGET_MODEL}")

[embedding_adapters] Fetching remote registry from https://raw.githubusercontent.com/PotentiallyARobot/embedding-adapters-registry/main/registry.json
[embedding_adapters] Successfully loaded remote registry and cached at /usr/local/lib/python3.12/dist-packages/models/registry_cache/registry.json
[embedding_adapters] Downloading adapter 'emb_adapter_minilm_to_openai_text-embedding-3-large_v1' from repo 'TylerF/emb_adapter_minilm_to_openai_text-embedding-3-large_v1' to: /usr/local/lib/python3.12/dist-packages/models/emb_adapter_minilm_to_openai_text-embedding-3-large_v1


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

(…)to_openai_text-embedding-3-large_v1.json:   0%|          | 0.00/193 [00:00<?, ?B/s]

LICENSE: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

adapter_quality_stats_minilm_to_openai_t(…):   0%|          | 0.00/48.4M [00:00<?, ?B/s]

adapter_minilm_to_openai_text-embedding-(…):   0%|          | 0.00/692M [00:00<?, ?B/s]

[EmbeddingAdapter] Loaded quality stats from /usr/local/lib/python3.12/dist-packages/models/emb_adapter_minilm_to_openai_text-embedding-3-large_v1/adapter_quality_stats_minilm_to_openai_text-embedding-3-large_v1.npz
Loaded adapter: sentence-transformers/all-MiniLM-L6-v2 → openai/text-embedding-3-small


### 3.3 Encode & Translate

In [25]:
# Sample texts to embed
texts = [
    "NASA announces discovery of Earth-like exoplanet.",
    "How do I build a RAG pipeline?",
    "The quick brown fox jumps over the lazy dog.",
    "Where can I find the best pizza in Washington D.C.?",
    "What restaurant has deep dish pizza in Washington D.C.?",
]

# Step 1: Generate source embeddings (normalized to match adapter training setup)
start = time.time()

src_embeddings = src_model.encode(
    texts,
    convert_to_numpy=True,
    normalize_embeddings=True,  # Important: matches adapter training setup
)

# Step 2: Translate to target space
translated_embeddings = adapter.encode_embeddings(src_embeddings)

elapsed_ms = (time.time() - start) * 1000

print(f"\n✅ Translated {len(texts)} embeddings in {elapsed_ms:.2f} ms")
print(f"   Average: {elapsed_ms / len(texts):.2f} ms per embedding")
print(f"\nSource shape:     {src_embeddings.shape}")
print(f"Translated shape: {translated_embeddings.shape}")


✅ Translated 5 embeddings in 17.76 ms
   Average: 3.55 ms per embedding

Source shape:     (5, 384)
Translated shape: (5, 1536)


### 3.4 Inspect the Results

In [23]:
# View first 8 dimensions of translated embeddings
for i, text in enumerate(texts):
    print(f"\nText: \"{text[:50]}...\"")
    print(f"  First 8 dims: {translated_embeddings[i][:8]}")


Text: "NASA announces discovery of Earth-like exoplanet...."
  First 8 dims: [-0.02015219 -0.029815   -0.02003191  0.00570335  0.00285336  0.01528635
 -0.01628077  0.01461365]

Text: "How do I build a RAG pipeline?..."
  First 8 dims: [ 0.02272716  0.00379894  0.00405332 -0.03134996 -0.03544979 -0.02624673
 -0.02306672 -0.03986255]

Text: "The quick brown fox jumps over the lazy dog...."
  First 8 dims: [-0.02005787  0.00247312  0.02189307 -0.04284604 -0.0156842   0.01917473
  0.02038561  0.01647771]

Text: "Where can I find the best pizza in Washington D.C...."
  First 8 dims: [-0.01189883 -0.02655029  0.00385452  0.03867225  0.0274068  -0.03793454
  0.01266782  0.05449203]

Text: "Where can I find the deep dish pizza in Washington..."
  First 8 dims: [-0.00494743 -0.0390961  -0.00088703  0.04625957  0.0052576  -0.0256073
  0.02638703  0.04561325]


---

## 4. Compute Similarity in Target Space

Translated embeddings live in the target space, so you can compute similarities just like you would with native target embeddings:

In [26]:
from numpy.linalg import norm

def cosine_similarity(a, b):
    """Compute cosine similarity between two vectors."""
    return np.dot(a, b) / (norm(a) * norm(b))

# Compare similarity between texts
print("Cosine Similarities (in target space):")
print("-" * 50)

for i in range(len(texts)):
    for j in range(i + 1, len(texts)):
        sim = cosine_similarity(translated_embeddings[i], translated_embeddings[j])
        print(f"[{i}] vs [{j}]: {sim:.4f}")
        print(f"    \"{texts[i][:40]}...\"")
        print(f"    \"{texts[j][:40]}...\"")
        print()

Cosine Similarities (in target space):
--------------------------------------------------
[0] vs [1]: -0.0729
    "NASA announces discovery of Earth-like e..."
    "How do I build a RAG pipeline?..."

[0] vs [2]: 0.0052
    "NASA announces discovery of Earth-like e..."
    "The quick brown fox jumps over the lazy ..."

[0] vs [3]: 0.0356
    "NASA announces discovery of Earth-like e..."
    "Where can I find the best pizza in Washi..."

[0] vs [4]: 0.0124
    "NASA announces discovery of Earth-like e..."
    "What restaurant has deep dish pizza in W..."

[1] vs [2]: -0.0209
    "How do I build a RAG pipeline?..."
    "The quick brown fox jumps over the lazy ..."

[1] vs [3]: 0.0694
    "How do I build a RAG pipeline?..."
    "Where can I find the best pizza in Washi..."

[1] vs [4]: 0.0407
    "How do I build a RAG pipeline?..."
    "What restaurant has deep dish pizza in W..."

[2] vs [3]: 0.0682
    "The quick brown fox jumps over the lazy ..."
    "Where can I find the best pizza in

---

## 5. (Optional) Quality Scoring

EmbeddingAdapters can help you detect when inputs are **out-of-distribution** (OOD) for the adapter. This is useful for:
- Debugging retrieval issues
- Deciding when to route to a more expensive model
- Understanding adapter limitations

In [30]:
from embedding_adapters.quality import interpret_quality

# Include some "unusual" text to see quality scoring in action
test_texts = [
    "NASA announces discovery of Earth-like exoplanet.",  # Normal
    "How do I build a RAG pipeline?",                     # Normal
    "🚀🎉💡🔥✨🌟🎊🎁",                                        # Emojis only
]

# Generate source embeddings
test_src_embs = src_model.encode(
    test_texts,
    convert_to_numpy=True,
    normalize_embeddings=True,
)

# Score quality (uses adapter's stored quality stats)
scores = adapter.score_source(test_src_embs)

# Human-readable interpretation
print("Quality Interpretation:")
print("=" * 60)
print(interpret_quality(test_texts, scores, space_label="source"))

Quality Interpretation:
Adapter quality interpretation (source space)
Number of examples: 3

Example 1: 'NASA announces discovery of Earth-like exoplanet.'
  - Confidence: 0.993  (very in-distribution)
  - Distances: Mahalanobis = 15.205, kNN distance = 0.892
  - Interpretation: This query looks very similar to the adapter's training data. The adapter is expected to behave reliably here.

Example 2: 'How do I build a RAG pipeline?'
  - Confidence: 0.968  (very in-distribution)
  - Distances: Mahalanobis = 15.716, kNN distance = 1.087
  - Interpretation: This query looks very similar to the adapter's training data. The adapter is expected to behave reliably here.

Example 3: '🚀🎉💡🔥✨🌟🎊🎁'
  - Confidence: 0.000  (strongly out-of-distribution)
  - Distances: Mahalanobis = 19.810, kNN distance = 1.112
  - Interpretation: This query is strongly out-of-distribution. The adapter is unlikely to behave reliably for this input.

Batch summary:
  - Mean confidence: 0.654 (range: 0.000 – 0.993)
  - M

---

## 6. Discover Available Adapters

You can explore what adapters are available:

In [31]:
# List available adapters (if using CLI)
!embedding-adapters list 2>/dev/null || echo "Run 'embedding-adapters list' in terminal to see available adapters"

[embedding_adapters] Fetching remote registry from https://raw.githubusercontent.com/PotentiallyARobot/embedding-adapters-registry/main/registry.json
[embedding_adapters] Successfully loaded remote registry and cached at /usr/local/lib/python3.12/dist-packages/models/registry_cache/registry.json
[
  {
    "source": "gemini/text-embedding-004",
    "target": "intfloat/e5-base-v2",
    "count": 1,
    "adapters": [
      {
        "slug": "emb_adapter_gemini_text_embedding_004_to_e5-base-v2-to_linear_v1",
        "flavor": "generic",
        "pro": true
      }
    ]
  },
  {
    "source": "gemini/text-embedding-004",
    "target": "openai/text-embedding-3-small",
    "count": 1,
    "adapters": [
      {
        "slug": "emb_adapter_gemini-text-embedding-004_to_text-embedding-3-small_linear_v1",
        "flavor": "generic",
        "pro": true
      }
    ]
  },
  {
    "source": "intfloat/e5-base-v2",
    "target": "gemini/text-embedding-004",
    "count": 1,
    "adapters": [
      {


In [32]:
# Show available source → target paths
!embedding-adapters paths 2>/dev/null || echo "Run 'embedding-adapters paths' in terminal to see available paths"

[embedding_adapters] Fetching remote registry from https://raw.githubusercontent.com/PotentiallyARobot/embedding-adapters-registry/main/registry.json
[embedding_adapters] Successfully loaded remote registry and cached at /usr/local/lib/python3.12/dist-packages/models/registry_cache/registry.json
[
  {
    "source": "gemini/text-embedding-004",
    "target": "intfloat/e5-base-v2",
    "count": 1,
    "slugs": [
      "emb_adapter_gemini_text_embedding_004_to_e5-base-v2-to_linear_v1"
    ]
  },
  {
    "source": "gemini/text-embedding-004",
    "target": "openai/text-embedding-3-small",
    "count": 1,
    "slugs": [
      "emb_adapter_gemini-text-embedding-004_to_text-embedding-3-small_linear_v1"
    ]
  },
  {
    "source": "intfloat/e5-base-v2",
    "target": "gemini/text-embedding-004",
    "count": 1,
    "slugs": [
      "emb_adapter_e5-base-v2_to_gemini_text_embedding_004_small_v1"
    ]
  },
  {
    "source": "intfloat/e5-base-v2",
    "target": "openai/text-embedding-3-small",
 

---

## 7. Example: Query an Existing Index

A common use case is querying a corpus that was embedded with an expensive cloud model, using a cheap local model + adapter:

In [33]:
# Simulated corpus (in production, this would be your vector DB)
corpus = [
    "Python is a popular programming language for data science.",
    "Machine learning models require large amounts of training data.",
    "Vector databases store embeddings for semantic search.",
    "RAG combines retrieval with language model generation.",
    "The capital of France is Paris.",
]

# Simulate: corpus was embedded with the target model (OpenAI)
# In reality, you'd have these stored in your vector DB
corpus_embeddings = src_model.encode(
    corpus,
    convert_to_numpy=True,
    normalize_embeddings=True,
)
corpus_translated = adapter.encode_embeddings(corpus_embeddings)

print(f"Corpus: {len(corpus)} documents embedded")

Corpus: 5 documents embedded


In [35]:
def search(query: str, top_k: int = 3):
    """Search the corpus using adapter-translated query embedding."""
    # Embed query with local model
    query_emb = src_model.encode(
        [query],
        convert_to_numpy=True,
        normalize_embeddings=True,
    )

    # Translate to target space
    query_translated = adapter.encode_embeddings(query_emb)[0]

    # Compute similarities
    similarities = [
        cosine_similarity(query_translated, doc_emb)
        for doc_emb in corpus_translated
    ]

    # Rank results
    ranked = sorted(enumerate(similarities), key=lambda x: x[1], reverse=True)

    return [(corpus[idx], score) for idx, score in ranked[:top_k]]


# Try a query
query = "What should I use to store vectors for search?"
results = search(query)

print(f"Query: \"{query}\"\n")
print("Top results:")
for i, (doc, score) in enumerate(results, 1):
    print(f"  {i}. [{score:.4f}] {doc}")

Query: "What should I use to store vectors for search?"

Top results:
  1. [0.6497] Vector databases store embeddings for semantic search.
  2. [0.3361] RAG combines retrieval with language model generation.
  3. [0.2169] Machine learning models require large amounts of training data.


---

## 8. Summary 🧠 → 🧠

You've learned how to:

| Task | Code |
|------|------|
| Load a source model | `SentenceTransformer(model_name)` |
| Load an adapter | `EmbeddingAdapter.from_registry(source, target, flavor)` |
| Generate embeddings | `src_model.encode(texts, normalize_embeddings=True)` |
| Translate embeddings | `adapter.encode_embeddings(src_embs)` |
| Score quality | `adapter.score_source(src_embs)` |

### Key Points

- **Always normalize** source embeddings (`normalize_embeddings=True`) to match adapter training
- Translated embeddings are compatible with your target embedding space
- Quality scoring helps detect out-of-distribution inputs
- Adapters run locally — no API calls, no latency, no cost per embedding

---

## Next Steps

- 📚 [Full Documentation](https://github.com/PotentiallyARobot/EmbeddingAdapters)
- 🔍 Explore available adapters: `embedding-adapters list`
- 🧪 Evaluate on your own data before production use
- 💬 Open an issue or PR for bugs, ideas, or new adapter requests